In [1]:
library(INLA)
library(rSPDE)
library(readr)
library(fmesher)
library(inlabru)
data_ori <- as.data.frame(read_csv("tmp/1180_vol.csv"))
logpt<-log(data_ori$Close)
logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)]
date <- data_ori$Date[1:(length(data_ori$Date)-1)]
   
date_timeindex<-as.numeric(as.POSIXct(date)) 
date_timeindex <- (date_timeindex - min(date_timeindex))/86400
mesh <- fm_mesh_1d(date_timeindex)
data = list(logreturn = logreturn, times = date_timeindex)


#OU model
OU_model <- rspde.matern(mesh = mesh, nu=0.5, prior.range.nominal = 50)
formula_OU <- logreturn ~ Intercept(1) +  OU(times, model = OU_model)
fit_OU <- bru(formula_OU, family="stochvol", data = data)

rspde_model <- rspde.matern(mesh = mesh, nu.upper.bound = 2, prior.range.nominal = 50)
formula_general <- logreturn ~ Intercept(1) + matern(times, model = rspde_model) 
fit_general <- bru(formula_general, family="stochvol", data = data)

train <- test <- list()
est.vol <- c() 
k = 1
for(i in seq(from = 10, to = length(date_timeindex)-10, by = 10)){
  train[[k]] <- 1:i
  test[[k]] <- (i+1):(i+10)
  est.vol <- c(est.vol, sqrt(var(logreturn[(i+1):(i+10)])))
  k = k+1
}


models <- list(fit_OU=fit_OU, fit_general=fit_general)

res2 <- cross_validation(models,train_test_indexes = list(train = train, test = test))


Loading required package: Matrix



Loading required package: sp

This is INLA_24.04.01 built 2024-04-01 19:43:22 UTC.
 - See www.r-inla.org/contact-us for how to get help.
 - List available models/likelihoods/etc with inla.list.models()
 - Use inla.doc(<NAME>) to access documentation

This is rSPDE 2.3.3.9000
- See https://davidbolin.github.io/rSPDE for vignettes and manuals.

Rows: 2372 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (9): Open, High, Low, Close, Volume, Dividends, Stock Splits, vol10, vol30
date (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [2]:
res2 <- cross_validation(models,train_test_indexes = list(train = train[1:10], test = test[1:10]))

Fold: 1 / 10 
Model: fit_OU 
Generating samples...
Samples generated!
MSE: 0.000328179402793863 
DSS: -5.70132478049813 
CRPS: -0.0119544662398515 
SCRPS: -0.748279685381048 
Fold: 1 / 10 
Model: fit_general 
Generating samples...
Samples generated!
MSE: 0.000367015708844511 
DSS: -5.83089556887779 
CRPS: -0.0120133414930712 
SCRPS: -0.759528435477684 
Fold: 2 / 10 
Model: fit_OU 
Generating samples...
Samples generated!
MSE: 0.00171609648745947 
DSS: -5.41405039776554 
CRPS: -0.0224080721618007 
SCRPS: -0.545347118766032 
Fold: 2 / 10 
Model: fit_general 
Generating samples...
Samples generated!
MSE: 0.00171451396644025 
DSS: -5.25560971416279 
CRPS: -0.0224819608517082 
SCRPS: -0.522056046713027 
Fold: 3 / 10 
Model: fit_OU 
Generating samples...
Samples generated!
MSE: 0.000429287030008779 
DSS: -6.37975440747405 
CRPS: -0.0125655163725497 
SCRPS: -0.820477666161541 
Fold: 3 / 10 
Model: fit_general 
Generating samples...
Samples generated!
MSE: 0.000431432792034187 
DSS: -6.3520998

In [3]:
res2

,Model,dss,mse,crps,scrps
,<chr>,<chr>,<chr>,<chr>,<chr>
1,fit_OU,-6.52264698792932,0.000496955185930569,-0.0115759013488574,-0.875604604854668
2,fit_general,-6.55264725341361,0.000499209730341885,-0.0114832570829678,-0.880799110586872
,Best,fit_general,fit_OU,fit_OU,fit_general


In [2]:
packageVersion("INLA")

[1] ‘24.4.1’

In [3]:
packageVersion("inlabru")

[1] ‘2.10.1’

In [4]:
packageVersion("rSPDE")

[1] ‘2.3.3’

In [5]:
res2

,Model,dss,mse,crps,scrps
,<chr>,<chr>,<chr>,<chr>,<chr>
1,fit_OU,0,0,0,0
2,fit_general,0,0,0,0
,Best,fit_OU,fit_OU,fit_OU,fit_OU


In [8]:
res2

,Model,dss,mse,crps,scrps
,<chr>,<chr>,<chr>,<chr>,<chr>
1,Model 1,0,0,0,0
,Best,Model 1,Model 1,Model 1,Model 1


In [3]:
cross_result[1]

,Model,dss,mse,crps,scrps
,<chr>,<chr>,<chr>,<chr>,<chr>
1,model,0,0,0,0
,Best,model,model,model,model
